In [1]:
import sqlite3


In [3]:
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()
print("Opened database successfully")

Opened database successfully


1. Which team scored the most points when playing at home?  

REAL Madrid

In [6]:
cursor.execute("""SELECT team.team_long_name, team_id, sum(home_goals) as home_goals_sum
FROM(
SELECT match.home_team_api_id as team_id, SUM(match.home_team_goal) AS home_goals
FROM Match
GROUP BY team_id
)
LEFT JOIN
Team
ON team_id =Team.team_api_id
GROUP BY team_id
ORDER BY home_goals_sum DESC
LIMIT 1;""")

In [7]:
cursor.fetchone()

('Real Madrid CF', 8633, 505)

2. Did this team also score the most points when playing away?  

NO. FC Barcelona has the most away goals

In [8]:
cursor.execute("""SELECT team.team_long_name, team_id, SUM(away_goals) as away_goals_sum
FROM(
SELECT match.away_team_api_id as team_id, SUM(match.away_team_goal) AS away_goals
FROM Match
GROUP BY team_id
)
LEFT JOIN
team
ON team_id = team.team_api_id
GROUP BY team_id
ORDER BY away_goals_sum DESC
LIMIT 1;"""
)

In [9]:
cursor.fetchone()

('FC Barcelona', 8634, 354)

3. How many matches resulted in a tie?  

6596

In [10]:
cursor.execute("""SELECT count(*)  as matches_resulting_in_tie
FROM Match
WHERE match.home_team_goal = match.away_team_goal;""")

In [11]:
cursor.fetchone()

(6596,)

4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

15 players with "Smith" in last name. 15 players with "Smith" anywhere in their name.

In [12]:
cursor.execute("""SELECT player.player_name FROM player;"""
)


In [13]:
rows = cursor.fetchall()


In [27]:
rows[0][0]

'Aaron Appindangoye'

In [31]:
last_name_smith_count = 0
for name in rows:
    names = name[0].lower().split(' ')
    if names[len(names) - 1] == "smith":
        last_name_smith_count += 1
last_name_smith_count

15

In [33]:
smith_name_count = 0
for name in rows:
    names = name[0].lower().split(' ')
    if "smith" in names:
        smith_name_count += 1
smith_name_count

15

5. What was the median tie score?

3

In [44]:
cursor.execute("""
SELECT COUNT(goal_count)  as num_rows, SUM(goal_count) as total_goals_from_rows
FROM (
SELECT home_team_goal as goal_count,  count(*)  as matches_resulting_in_tie 
FROM Match
WHERE match.home_team_goal = match.away_team_goal
GROUP BY goal_count
ORDER BY goal_count);"""
)

In [45]:
row = cursor.fetchall() 

In [46]:
row

[(7, 21)]

In [50]:
cursor.execute(f"""SELECT goal_count
FROM
(
SELECT home_team_goal as goal_count,  count(*)  as matches_resulting_in_tie
FROM Match
WHERE match.home_team_goal = match.away_team_goal
GROUP BY goal_count
ORDER BY goal_count
)
WHERE goal_count = {row[0][1]}/{row[0][0]}
GROUP BY goal_count;""")

In [51]:
cursor.fetchall()

[(3,)]

6. What percentage of players prefer their left or right foot?

left ~= 24% , right ~= 75%

In [56]:
cursor.execute("""SELECT COUNT(*) as total_players
FROM Player_Attributes;""")

In [57]:
total_player_count = cursor.fetchall()

In [58]:
total_player_count

[(183978,)]

In [64]:
cursor.execute("""SELECT COUNT(Player_Attributes.preferred_foot) as left_footed_count
FROM Player_Attributes
WHERE Player_Attributes.preferred_foot = 'left';""")

In [65]:
left_footed_count = cursor.fetchall()

In [69]:
left_footed_count[0][0] / total_player_count[0][0]

0.24314320190457556

In [70]:
cursor.execute("""SELECT COUNT(Player_Attributes.preferred_foot) as right_footed_count
FROM Player_Attributes
WHERE Player_Attributes.preferred_foot = 'right';""");

In [71]:
right_footed_count = cursor.fetchall()

In [72]:
right_footed_count[0][0] / total_player_count[0][0]

0.7523127765276283